The first section only have exploratory trials...

In [12]:
## This function will use CKAN api to get packages list plus the url for the resources under each entity 
## The results are stored in a pandas DF

import pprint
import requests
import pandas as pd

def getPackageList(url_api):
    r = requests.get(url_api)
    j = r.json()
    return(pd.DataFrame({"package_id": j['result']}))

def getResourceUrl(package_id):
    url = "https://opendata.porto.hackacity.eu" + "/api/3/action/package_show?id=" + package_id;  
    r = requests.get(url)
    j = r.json()
    return(j['result']['resources'][0]['url'])

In [ ]:
## Usage

url_api = "https://opendata.porto.hackacity.eu/api/3/action/package_list"
df = getPackageList(url_api)

df["resource_url"] = list(map(getResourceUrl, df["package_id"]))

#print(df)

In [4]:
## Get packageIds as a list

def getPackageIds(url):
    r = requests.get(url)
    j = r.json()
    return(j['result'])

In [ ]:
## Usage

url = "https://opendata.porto.hackacity.eu/api/3/action/package_list";
packageIdsList = getPackageIds(url)
#print(packageIdsList)

In [279]:
## For a list of packagesIds extract the available metadata and store the results in a pandas DF

from pandas.io.json import json_normalize

def normalizeJSON(url, package_id):
    url = url + package_id;  
    r = requests.get(url)
    j = r.json()
    return(json_normalize(j['result']))

def appendRowToDF(df1, df2):
    df = pd.DataFrame()
    if df1.empty: 
        df = df1.copy()
    else:
        df = pd.concat([df1,df2])
    return(df)

In [ ]:
## Usage

packagesMetadata = pd.DataFrame()
url = "https://opendata.porto.hackacity.eu/api/3/action/package_show?id="
for packageId in packageIdsList:
    row = normalizeJSON(url, packageId)
    packagesMetadata = appendRowToDF(packagesMetadata, row)  

In [45]:
## Request live data and store the result in a panda DF

import requests
import pprint
import pandas
from pandas.io.json import json_normalize

def getLiveData(api_url, package_id = None, device_id = None):
    if (package_id is None) and (device_id is not None):
        url = api_url + "/v2/entities?id=" + device_id
    elif (package_id is not None) and (device_id is None):
        url = api_url + "/v2/entities?type=" + package_id
    else:
        raise Exception("package_id or device_id required or are simultaneous defined!")

    print(url)
    
    r = requests.get(url)
    j = r.json()
    #pprint.pprint(j)
    return(json_normalize(j))

In [256]:
## Example with error
#api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
#df = getLiveData(api_url)

In [46]:
## Example without device id
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
package_id = "AirQualityObserved"
df = getLiveData(api_url, package_id)

https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?type=AirQualityObserved


In [51]:
## Example with device id
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
device_id = "urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54"
df = getLiveData(api_url, device_id = device_id)

https://broker.fiware.urbanplatform.portodigital.pt/v2/entities?id=urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54


In [14]:
## Example 
## require getHistoricalData function declared below!!!

#resource_list = ['urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54', 
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1bde521e3053085c08c', 
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b632f2706599b05e998bed8',
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b72dbfa06599b05e9a74f27',
#                 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1a6e521e3053085c067']

#api_url = "http://history-data.urbanplatform.portodigital.pt"

#df = pd.DataFrame()
#for resource in resource_list:
#    print(resource)
#    df1 = getHistoricalData(api_url, device_id = resource, no=500)
#    #print(df1.columns)
#    if df.empty:
#        df = df1.copy()
#    else:
#        df = pd.concat([df,df1])
#    #print(df.columns)   


From this point forward are the sequential steps to get the IOT data...


In [306]:
## List IOT entities
import requests
import json

def getListIOTEntities(api_url):
    url = api_url + "/v2/types"
    r = requests.get(url)
    j = r.json()
    entity_list = []
    for item in j:
        #print(item['type'])
        entity_list.append(item['type'])
    return(entity_list)

In [307]:
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"
entity_list = getListIOTEntities(api_url)
print(entity_list)

['AirQualityObserved', 'ArrivalEstimation', 'Device', 'Event', 'GtfsTransitFeedFile', 'NoiseLevelObserved', 'OffStreetParking', 'PointOfInterest', 'TrafficFlowObserved', 'Vehicle', 'WeatherForecast', 'WeatherObserved', 'gtfsAgency', 'gtfsCalendarDateRule', 'gtfsCalendarRule', 'gtfsRoute', 'gtfsService', 'gtfsStop']


In [338]:
## Reduce entity_list to the ones of interest (MANUAL STEP)
entity_list = ['AirQualityObserved', 'NoiseLevelObserved']

In [332]:
## Get list of all available resources/devices for an entity

import requests
import pprint

def getResourceList(api_url, entity_name):
    # First let's the ID's from the FiWare
    url = api_url + "/v2/entities?type=" + entity_name
    #print(url)
    r = requests.get(url, verify=False)
    j = r.json()
    #print(j)
    resources_list = []
    for resource in j: 
      resources_list.append(resource['id'])
    return(resources_list)

In [339]:
api_url = "https://broker.fiware.urbanplatform.portodigital.pt"

resource_list = []
for entity in entity_list:
    #print(entity)
    resource_list.append(getResourceList(api_url, entity))    

/home/cha1av/dev/hackacity2019/env/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/cha1av/dev/hackacity2019/env/lib/python3.5/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [ ]:
#print(resource_list)

In [340]:
#Flat resource_list
resource_list_flat = [item for sublist in resource_list for item in sublist]

In [341]:
#Remove stuff that is giving errors
resource_list_flat.remove('Trindade_1_Rotacao')

ValueError: list.remove(x): x not in list

In [342]:
print(resource_list_flat)

['urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1bde521e3053085c08c', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b632f2706599b05e998bed8', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b72dbfa06599b05e9a74f27', 'urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1a6e521e3053085c067', 'urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af0280de8eb4bab42510c', 'urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af0d70de8eb4bab425125', 'urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5aafa44d4a62a76dc692944a', 'urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af1d40de8eb4bab4251a9', 'urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af1a50de8eb4bab4251a7', 'urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af1fb0de8eb4bab4251aa', 'urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8ea72b1ca835782ea71a87', 'urn:ngsi

In [343]:
## Request historical data with resource_id and store the results as a panda DF for a single device

import requests
import pprint
import pandas
from pandas.io.json import json_normalize

def convertJSONHistoricalToDF(j):
    df = pd.DataFrame()
    for i in range(0,len(j['data']['attributes'])):
        df[j['data']['attributes'][i]['attrName']] = j['data']['attributes'][i]['values']
    ## expand location attributes
    for key in df['location'][0].keys():
        df[key] = str(df['location'][0][key])
    df['device_id'] = j['data']['entityId']
    df['indexT'] = j['data']['index']
    #print(df)
    return(df)
    
def getHistoricalData(api_url, device_id = None, no = 20):
    if (device_id is not None):
        url = api_url + "/v2/entities/" + device_id + "?limit=" + str(no)
    else:
        raise Exception("device_id is required")

    #print(url)
    r = requests.get(url)
    j = r.json()
    #json_normalize(j['data'], record_path=['attributes']).transpose()
    return(convertJSONHistoricalToDF(j))

In [344]:
api_url = "http://history-data.urbanplatform.portodigital.pt"

df = pd.DataFrame()
for resource in resource_list_flat:
    print(resource)
    df1 = getHistoricalData(api_url, device_id = resource, no=500)
    #print(df1.columns)
    if df.empty:
        df = df1.copy()
    else:
        df = pd.concat([df,df1], sort=False)
    #print(df.columns)  

urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5adf39366f555a4514e7ea54
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1bde521e3053085c08c
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b632f2706599b05e998bed8
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b72dbfa06599b05e9a74f27
urn:ngsi-ld:AirQualityObserved:porto:environment:ubiwhere:5b97a1a6e521e3053085c067
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af0280de8eb4bab42510c
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af0d70de8eb4bab425125
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5aafa44d4a62a76dc692944a
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af1d40de8eb4bab4251a9
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af1a50de8eb4bab4251a7
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8af1fb0de8eb4bab4251aa
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubiwhere:5a8ea72b1ca835782ea71a87
urn:ngsi-ld:NoiseLevelObserved:porto:noise:ubi

In [351]:
pprint.pprint(df)

        CO    NO2    O3     Ox   PM1  PM10  PM25             dateObserved  \
0    461.7  110.2   6.8  120.8  20.0  67.3  50.0  2018-06-29T19:02:41.000   
1    461.7  110.2   6.8  120.8  19.8  70.4  49.5  2018-06-29T19:07:13.000   
2    353.2   97.3  12.3  112.8  22.2  69.5  54.4  2018-06-29T19:12:45.000   
3    353.2   97.3  12.3  112.8  22.8  73.7  56.0  2018-06-29T19:17:17.000   
4    353.2   97.3  12.3  112.8  21.6  83.6  53.6  2018-06-29T19:22:49.000   
5    446.0   83.5  20.1  107.2  19.2  60.5  48.5  2018-06-29T19:27:21.000   
6    446.0   83.5  20.1  107.2  18.8  66.0  47.0  2018-06-29T19:31:53.000   
7    446.0   83.5  20.1  107.2  18.4  66.0  45.6  2018-06-29T19:37:25.000   
8    355.0   85.7  20.0  109.4  16.8  59.4  42.4  2018-06-29T19:41:57.000   
9    355.0   85.7  20.0  109.4  18.6  58.5  45.6  2018-06-29T19:47:29.000   
10   355.0   85.7  20.0  109.4  17.8  64.9  45.0  2018-06-29T19:52:01.000   
11   419.3   87.6  21.1  112.5  15.6  52.8  39.6  2018-06-29T19:57:33.000   